In [131]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
from scipy import stats  # Add this import for the t-distribution

In [132]:

filepath = '/Users/danielseymour/Developer/EC334-Summative/raw_data/productivity_data/total_factor_productivity.csv'

tfp_disp_df = pd.read_csv(filepath, encoding='latin1')

In [133]:
tfp_disp_df.head()

,naics4,year,sd,d7525,d9010,d9990,d1001,sd*,d7525*,d9010*,d9990*,d1001*
0,3111,1987.0,0.2883,0.2239,0.6477,0.6196,0.1455,0.2789,0.1947,0.4894,0.8043,0.1535
1,3111,1988.0,0.2782,0.1996,0.5330,0.8425,0.1384,0.3102,0.1529,0.5752,0.8945,0.1245
2,3111,1989.0,0.3253,0.1840,0.7220,0.7709,0.1533,0.3094,0.1547,0.5261,0.8397,0.1173
3,3111,1990.0,0.2954,0.2001,0.5154,1.037,0.1929,0.2927,0.1822,0.4714,0.7978,0.1004
4,3111,1991.0,0.2842,0.2235,0.5372,0.6796,0.2426,0.2842,0.1806,0.4711,0.7504,0.2352


In [134]:

filepath = '/Users/danielseymour/Developer/EC334-Summative/raw_data/productivity_data/total-factor-productivity-manufacturing-and-transportation-detailed-industries.xlsx'

manufacturing_tfp = pd.read_excel(filepath)


In [135]:
# Skip the first two rows of the already loaded DataFrame
manufacturing_tfp = manufacturing_tfp.iloc[1:].reset_index(drop=True)

In [136]:
manufacturing_tfp.head()

,"Annual total factor productivity and related measures for detailed industries, N.A. = data not available",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,Sector,NAICS,IndustryTitle,Digit,Basis,Measure,Units,1987,1988.000,1989.000,...,2015.000,2016.000,2017.0,2018.000,2019.000,2020.000,2021.000,2022,2023,2024
1,Manufacturing,323,Printing and related support activities,3-Digit,All workers,Total factor productivity,Index (2017=100),92.284,91.297,91.134,...,96.861,99.356,100.0,103.018,98.974,98.353,100.457,N.A.,N.A.,N.A.
2,Manufacturing,323,Printing and related support activities,3-Digit,All workers,Labor productivity,Index (2017=100),68.989,67.071,66.388,...,96.476,98.962,100.0,101.347,96.510,103.967,100.948,98.834,95.481,97.372
3,Manufacturing,323,Printing and related support activities,3-Digit,All workers,Real sectoral output,Index (2017=100),119.875,123.533,123.913,...,98.644,100.499,100.0,100.019,94.110,85.437,85.864,84.766,79.12,78.39
4,Manufacturing,323,Printing and related support activities,3-Digit,All workers,Output per worker,Index (2017=100),69.105,67.285,66.877,...,96.371,99.445,100.0,102.685,97.837,100.561,101.400,98.172,93.7,95.932


In [137]:
import pandas as pd
import re

# Since the headers are in row 0, don't skip any rows when loading
# If manufacturing_tfp is already loaded, we'll make a fresh copy
# If it's not loaded properly, reload it without skipping rows
try:
    # Check if manufacturing_tfp exists and has data
    if 'manufacturing_tfp' in locals() and len(manufacturing_tfp) > 0:
        print("Using existing manufacturing_tfp DataFrame")
    else:
        raise NameError("manufacturing_tfp not properly loaded")
except:
    # If needed, reload from file without skipping rows
    print("Reloading data from file...")
    tfp_file_path = 'path_to_your_file.xlsx'  # Replace with your actual file path
    manufacturing_tfp = pd.read_excel(tfp_file_path)

# Print what we're working with
print(f"Original DataFrame shape: {manufacturing_tfp.shape}")
print("First few rows:")
print(manufacturing_tfp.head())

# Check if 'NAICS' is in the columns
if 'NAICS' in manufacturing_tfp.columns:
    print("'NAICS' column found in headers")
else:
    # If 'NAICS' is not in columns, it might be in the first row
    print("'NAICS' not in column headers, checking first row...")
    if 'NAICS' in manufacturing_tfp.iloc[0].values:
        print("'NAICS' found in first row, using as header")
        # Get the index of the 'NAICS' value in the first row
        naics_idx = list(manufacturing_tfp.iloc[0]).index('NAICS')
        print(f"'NAICS' found at index {naics_idx}")
        
        # Set the headers from the first row
        manufacturing_tfp.columns = manufacturing_tfp.iloc[0]
        # Remove the first row (now redundant)
        manufacturing_tfp = manufacturing_tfp.iloc[1:].reset_index(drop=True)
        print("Headers set from first row")

# Now check again if 'NAICS' is in columns
if 'NAICS' in manufacturing_tfp.columns:
    print("Now working with 'NAICS' column")
else:
    # If still not found, print all columns for manual selection
    print("NAICS column not found in headers or first row")
    print("Available columns:")
    for i, col in enumerate(manufacturing_tfp.columns):
        print(f"{i}: {col}")
    
    # Manually select the NAICS column index based on visual inspection
    naics_col_idx = 1  # Change this to the correct index based on your data
    naics_col = manufacturing_tfp.columns[naics_col_idx]
    print(f"Using column at index {naics_col_idx}: '{naics_col}' as NAICS column")
    
    # Rename this column to 'NAICS' for consistency
    col_mapping = {naics_col: 'NAICS'}
    manufacturing_tfp = manufacturing_tfp.rename(columns=col_mapping)
    print("Renamed column to 'NAICS'")

# Now we can safely filter for 4-digit NAICS codes
# First convert NAICS to string
manufacturing_tfp['NAICS'] = manufacturing_tfp['NAICS'].astype(str)

# Function to check if a NAICS code has 4 digits
def is_four_digit_naics(naics):
    # Remove any non-numeric characters
    digits = re.sub(r'\D', '', str(naics))
    # Check if it starts with 4 digits
    return len(digits) >= 4 and len(digits.split('.')[0]) == 4

# Filter for 4-digit NAICS codes
four_digit_tfp = manufacturing_tfp[manufacturing_tfp['NAICS'].apply(is_four_digit_naics)]
print(f"\nAfter filtering for 4-digit NAICS: {len(four_digit_tfp)} rows")

Using existing manufacturing_tfp DataFrame
Original DataFrame shape: (7867, 45)
First few rows:
  Annual total factor productivity and related measures for detailed industries, N.A. = data not available  \
0                                             Sector                                                         
1                                      Manufacturing                                                         
2                                      Manufacturing                                                         
3                                      Manufacturing                                                         
4                                      Manufacturing                                                         

  Unnamed: 1                               Unnamed: 2 Unnamed: 3   Unnamed: 4  \
0      NAICS                            IndustryTitle      Digit        Basis   
1        323  Printing and related support activities    3-Digit  All workers   


In [138]:
# Check if there's a Measure column to filter for TFP
if 'Measure' in manufacturing_tfp.columns:
    print("\nMeasure column found")
    print("Available measures:")
    print(manufacturing_tfp['Measure'].unique())
    
    # Filter for Total factor productivity
    if 'Total factor productivity' in manufacturing_tfp['Measure'].values:
        tfp_only = four_digit_tfp[four_digit_tfp['Measure'] == 'Total factor productivity']
        print(f"Filtered for Total factor productivity: {len(tfp_only)} rows")
    else:
        # Look for similar names
        tfp_measures = [m for m in manufacturing_tfp['Measure'].unique() 
                        if 'factor' in str(m).lower() and 'productiv' in str(m).lower()]
        if tfp_measures:
            print(f"Found similar measures: {tfp_measures}")
            tfp_only = four_digit_tfp[four_digit_tfp['Measure'].isin(tfp_measures)]
            print(f"Filtered for TFP-like measures: {len(tfp_only)} rows")
        else:
            print("No Total factor productivity measure found")
            tfp_only = four_digit_tfp  # Keep all measures if TFP not found
else:
    print("No Measure column found, proceeding with all data")
    tfp_only = four_digit_tfp

# Identify year columns - looking for columns that might be years between 1980-2025
year_cols = []
for col in tfp_only.columns:
    if isinstance(col, (int, float)) and 1980 <= col <= 2025:
        year_cols.append(col)
    elif isinstance(col, str):
        # Try to extract year from string column names
        match = re.search(r'(19|20)\d{2}', col)
        if match:
            try:
                year = int(match.group(0))
                if 1980 <= year <= 2025:
                    year_cols.append(col)
            except ValueError:
                pass

print(f"\nIdentified {len(year_cols)} year columns: {year_cols[:5]}...")

# Reshape data to have years as rows
id_cols = [col for col in tfp_only.columns if col not in year_cols]
print(f"ID columns: {id_cols}")

if year_cols:
    # Reshape from wide to long
    tfp_long = pd.melt(
        tfp_only,
        id_vars=id_cols,
        value_vars=year_cols,
        var_name='year',
        value_name='tfp_value'
    )
    
    # Ensure year is numeric
    tfp_long['year'] = pd.to_numeric(tfp_long['year'], errors='coerce')
    
    # Drop rows with missing values
    tfp_long = tfp_long.dropna(subset=['tfp_value'])
    
    # Create simplified version
    simplified_tfp = tfp_long[['NAICS', 'year', 'tfp_value']].copy()
    simplified_tfp.columns = ['naics4', 'year', 'tfp']
    
    print(f"\nReshaped data: {len(tfp_long)} rows")
    print("First few rows:")
    print(tfp_long.head())
    
    print("\nSimplified version:")
    print(simplified_tfp.head())


Measure column found
Available measures:
['Total factor productivity' 'Labor productivity' 'Real sectoral output'
 'Output per worker' 'Unit labor costs' 'Capital input'
 'Intermediate inputs' 'Combined inputs' 'Capital productivity'
 'Intermediate inputs productivity' 'Employment' 'Hours worked'
 'Labor compensation' 'Hourly compensation'
 'Sectoral output price deflator' 'Capital intensity'
 'Contribution of capital intensity to labor productivity'
 'Contribution of intermediate inputs intensity to labor productivity'
 'Intermediate inputs intensity' 'Combined inputs price deflator'
 'Sectoral output' 'Capital costs' 'Intermediate inputs costs'
 'Combined inputs costs' 'Labor share' 'Capital share'
 'Intermediate inputs share']
Filtered for Total factor productivity: 172 rows

Identified 38 year columns: ['1987', np.float64(1988.0), np.float64(1989.0), np.float64(1990.0), np.float64(1991.0)]...
ID columns: ['Sector', 'NAICS', 'IndustryTitle', 'Digit', 'Basis', 'Measure', 'Units']

R

In [139]:
simplified_tfp.tail(10)

,naics4,year,tfp
6526,3371,2024.0,N.A.
6527,3371,2024.0,N.A.
6528,3372,2024.0,N.A.
6529,3372,2024.0,N.A.
6530,3379,2024.0,N.A.
6531,3379,2024.0,N.A.
6532,3391,2024.0,N.A.
6533,3391,2024.0,N.A.
6534,3399,2024.0,N.A.
6535,3399,2024.0,N.A.


In [140]:
filepath = '/Users/danielseymour/Developer/EC334-Summative/raw_data/dynamism_data/bds2022_vcn4.csv'

bds2022_vcn4 = pd.read_csv(filepath, encoding='latin1')

In [141]:
bds2022_vcn4.head() 

,year,vcnaics4,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,estabs_exit,estabs_exit_rate,...,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,1978,1131,193,200,3067,2948,42,21.483,33,16.880,...,474,311,16.081,26.633,255,8.651,53.265,22,22,37
1,1978,1132,84,100,1745,1762,12,11.707,17,16.585,...,42,546,2.384,33.381,-51,-2.895,60.971,10,10,25
2,1978,1133,12079,12196,98382,93837,2382,19.806,2043,16.987,...,7320,10648,7.801,19.148,9138,9.738,38.296,1521,1524,5347
3,1978,1141,1504,1540,10486,9559,383,25.008,366,23.898,...,1253,1344,13.108,27.168,1837,19.217,54.336,272,272,799
4,1978,1142,147,147,694,650,34,23.368,31,21.306,...,71,142,10.931,32.794,91,14.011,65.589,25,25,64


In [142]:
# Ensure both keys are strings for merging
tfp_disp_df['naics4'] = tfp_disp_df['naics4'].astype(str)
bds2022_vcn4['vcnaics4'] = bds2022_vcn4['vcnaics4'].astype(str)

merged_df = pd.merge(
	tfp_disp_df, 
	bds2022_vcn4, 
	how='inner', 
	left_on=['naics4', 'year'], 
	right_on=['vcnaics4', 'year']
)

In [143]:
merged_df.head()

,naics4,year,sd,d7525,d9010,d9990,d1001,sd*,d7525*,d9010*,...,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,3111,1987.0,0.2883,0.2239,0.6477,0.6196,0.1455,0.2789,0.1947,0.4894,...,1802,4282,3.717,12.548,-341,-0.703,23.690,72,75,619
1,3111,1988.0,0.2782,0.1996,0.5330,0.8425,0.1384,0.3102,0.1529,0.5752,...,3225,3817,6.665,14.553,-652,-1.347,26.411,76,77,1085
2,3111,1989.0,0.3253,0.1840,0.7220,0.7709,0.1533,0.3094,0.1547,0.5261,...,2015,4145,4.178,12.773,-1152,-2.389,20.768,70,70,704
3,3111,1990.0,0.2954,0.2001,0.5154,1.037,0.1929,0.2927,0.1822,0.4714,...,1567,3306,3.240,10.077,1012,2.093,20.153,76,77,690
4,3111,1991.0,0.2842,0.2235,0.5372,0.6796,0.2426,0.2842,0.1806,0.4711,...,2134,3494,4.321,11.397,-347,-0.703,21.388,61,63,745


In [144]:
merged_df.columns

Index(['naics4', 'year', 'sd', 'd7525', 'd9010', 'd9990', 'd1001', 'sd*',
       'd7525*', 'd9010*', 'd9990*', 'd1001*', 'vcnaics4', 'firms', 'estabs',
       'emp', 'denom', 'estabs_entry', 'estabs_entry_rate', 'estabs_exit',
       'estabs_exit_rate', 'job_creation', 'job_creation_births',
       'job_creation_continuers', 'job_creation_rate_births',
       'job_creation_rate', 'job_destruction', 'job_destruction_deaths',
       'job_destruction_continuers', 'job_destruction_rate_deaths',
       'job_destruction_rate', 'net_job_creation', 'net_job_creation_rate',
       'reallocation_rate', 'firmdeath_firms', 'firmdeath_estabs',
       'firmdeath_emp'],
      dtype='object')

In [145]:
# List of columns that should be numeric based on the aggregation dictionary
columns_to_convert = [
    # Dispersion measures
    'd9990', 'd1001', 'd9990*', 'd1001*',
    
    # Entry/exit counts
    'estabs_entry', 'estabs_exit',
    'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp',
    
    # Entry/exit rates  
    'estabs_entry_rate', 'estabs_exit_rate',
    
    # Job flow counts
    'job_creation_births', 'job_creation_continuers',
    'job_destruction_deaths', 'job_destruction_continuers',
    
    # Job flow rates
    'job_creation_rate_births', 'job_destruction_rate_deaths'
]

# Convert each column to numeric
print("Converting object columns to numeric...")
for col in columns_to_convert:
    if col in merged_df.columns and merged_df[col].dtype == 'object':
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
        nan_count = merged_df[col].isna().sum()
        if nan_count > 0:
            print(f"  {col}: converted ({nan_count} NaN values created)")
        else:
            print(f"  {col}: converted successfully")

Converting object columns to numeric...
  d9990: converted (17 NaN values created)
  d1001: converted (16 NaN values created)
  d9990*: converted (8 NaN values created)
  d1001*: converted (8 NaN values created)
  estabs_entry: converted successfully
  estabs_exit: converted successfully
  firmdeath_firms: converted (4 NaN values created)
  firmdeath_estabs: converted (4 NaN values created)
  firmdeath_emp: converted (4 NaN values created)
  estabs_entry_rate: converted successfully
  estabs_exit_rate: converted successfully
  job_creation_births: converted successfully
  job_creation_continuers: converted successfully
  job_destruction_deaths: converted successfully
  job_destruction_continuers: converted successfully
  job_creation_rate_births: converted successfully
  job_destruction_rate_deaths: converted successfully


In [146]:
# Create a clean copy to work with
df = merged_df.copy()

##############################################################################
# 1. CREATE TWO-YEAR WINDOWS
##############################################################################
BASE_YEAR = df['year'].min()
df = df.sort_values(['naics4', 'year'])

# Create window identifiers
df['window_id'] = ((df['year'] - BASE_YEAR) // 2).astype(int)
df['window_end_year'] = BASE_YEAR + df['window_id'] * 2 + 1

print(f"Years span from {df['year'].min()} to {df['year'].max()}")
print(f"This creates {df['window_id'].max() + 1} two-year windows")

Years span from 1987.0 to 2020.0
This creates 17 two-year windows


In [147]:
agg_dict = {
   # DISPERSION MEASURES - AVERAGE within 2-year window
   'sd': 'mean',                    # Standard deviation - average
   'd7525': 'mean',                 # IQR (75-25 percentile) - average
   'd9010': 'mean',                 # 90-10 percentile - average 
   'd9990': 'mean',                 # 99-90 percentile - average
   'd1001': 'mean',                 # 10-01 percentile - average
   'sd*': 'mean',                   # Weighted SD - average
   'd7525*': 'mean',                # Weighted IQR - average
   'd9010*': 'mean',                # Weighted 90-10 - average
   'd9990*': 'mean',                # Weighted 99-90 - average
   'd1001*': 'mean',                # Weighted 10-01 - average
  
   # STOCK VARIABLES - LAST value (end of period snapshot)
   'firms': 'last',                 # Number of firms - end of period
   'estabs': 'last',                # Number of establishments - end of period
   'emp': 'last',                   # Employment - end of period
   'denom': 'last',                 # Denominator for rates - end of period
  
   # ENTRY/EXIT COUNTS - SUM across 2 years
   'estabs_entry': 'sum',           # Count of entering establishments - total over 2 years
   'estabs_exit': 'sum',            # Count of exiting establishments - total over 2 years
   'firmdeath_firms': 'sum',        # Count of dying firms - total over 2 years
   'firmdeath_estabs': 'sum',       # Establishments from dying firms - total over 2 years
   'firmdeath_emp': 'sum',          # Employment from dying firms - total over 2 years
  
   # ENTRY/EXIT RATES - AVERAGE across 2 years
   'estabs_entry_rate': 'mean',    # Establishment entry rate - average annual rate
   'estabs_exit_rate': 'mean',     # Establishment exit rate - average annual rate
  
   # JOB FLOW COUNTS - SUM across 2 years
   'job_creation': 'sum',           # Total jobs created - sum over 2 years
   'job_creation_births': 'sum',    # Jobs from new establishments - sum over 2 years
   'job_creation_continuers': 'sum', # Jobs from continuing establishments - sum
   'job_destruction': 'sum',        # Total jobs destroyed - sum over 2 years
   'job_destruction_deaths': 'sum', # Jobs lost from deaths - sum over 2 years
   'job_destruction_continuers': 'sum', # Jobs lost from continuers - sum
   'net_job_creation': 'sum',      # Net job change - sum over 2 years
  
   # JOB FLOW RATES - AVERAGE across 2 years
   'job_creation_rate': 'mean',    # Job creation rate - average annual rate
   'job_creation_rate_births': 'mean', # Job creation rate from births - average
   'job_destruction_rate': 'mean', # Job destruction rate - average annual rate
   'job_destruction_rate_deaths': 'mean', # Destruction rate from deaths - average
   'net_job_creation_rate': 'mean', # Net job creation rate - average annual rate
   'reallocation_rate': 'mean',    # Job reallocation rate - average annual rate
}


# The 'vcnaics4' column is likely non-numeric, so we exclude it
# If you need it, you could add: 'vcnaics4': 'first' or 'last'


print("AGGREGATION RULES BY CATEGORY:")
print("="*60)


print("\n1. DISPERSION MEASURES (AVERAGE):")
print("   These are averaged because we want the typical dispersion level during the 2-year period")
for col in ['sd', 'd7525', 'd9010', 'd9990', 'd1001', 'sd*', 'd7525*', 'd9010*', 'd9990*', 'd1001*']:
   if col in agg_dict:
       print(f"   {col:12} -> {agg_dict[col]}")


print("\n2. STOCK VARIABLES (LAST VALUE):")
print("   These are end-of-period snapshots")
for col in ['firms', 'estabs', 'emp', 'denom']:
   if col in agg_dict:
       print(f"   {col:12} -> {agg_dict[col]}")


print("\n3. FLOW COUNTS (SUM):")
print("   These are cumulative flows over the 2-year period")
flow_counts = ['estabs_entry', 'estabs_exit', 'job_creation', 'job_creation_births',
              'job_creation_continuers', 'job_destruction', 'job_destruction_deaths',
              'job_destruction_continuers', 'net_job_creation', 'firmdeath_firms',
              'firmdeath_estabs', 'firmdeath_emp']
for col in flow_counts:
   if col in agg_dict:
       print(f"   {col:30} -> {agg_dict[col]}")


print("\n4. RATES (AVERAGE):")
print("   These are averaged to get typical annual rates during the period")
rate_vars = ['estabs_entry_rate', 'estabs_exit_rate', 'job_creation_rate',
            'job_creation_rate_births', 'job_destruction_rate',
            'job_destruction_rate_deaths', 'net_job_creation_rate', 'reallocation_rate']
for col in rate_vars:
   if col in agg_dict:
       print(f"   {col:30} -> {agg_dict[col]}")


print(f"\nTotal columns with aggregation rules: {len(agg_dict)}")

AGGREGATION RULES BY CATEGORY:

1. DISPERSION MEASURES (AVERAGE):
   These are averaged because we want the typical dispersion level during the 2-year period
   sd           -> mean
   d7525        -> mean
   d9010        -> mean
   d9990        -> mean
   d1001        -> mean
   sd*          -> mean
   d7525*       -> mean
   d9010*       -> mean
   d9990*       -> mean
   d1001*       -> mean

2. STOCK VARIABLES (LAST VALUE):
   These are end-of-period snapshots
   firms        -> last
   estabs       -> last
   emp          -> last
   denom        -> last

3. FLOW COUNTS (SUM):
   These are cumulative flows over the 2-year period
   estabs_entry                   -> sum
   estabs_exit                    -> sum
   job_creation                   -> sum
   job_creation_births            -> sum
   job_creation_continuers        -> sum
   job_destruction                -> sum
   job_destruction_deaths         -> sum
   job_destruction_continuers     -> sum
   net_job_creation            

In [148]:
two_yr = df.groupby(['naics4', 'window_id', 'window_end_year'], as_index=False).agg(agg_dict)

In [149]:
two_yr.head(10)

,naics4,window_id,window_end_year,sd,d7525,d9010,d9990,d1001,sd*,d7525*,...,job_destruction,job_destruction_deaths,job_destruction_continuers,net_job_creation,job_creation_rate,job_creation_rate_births,job_destruction_rate,job_destruction_rate_deaths,net_job_creation_rate,reallocation_rate
0,3111,0,1988.0,0.28325,0.21175,0.59035,0.73105,0.14195,0.29455,0.17380,...,13126,5027,8099,-993,12.5250,4.7275,13.5505,5.1910,-1.0250,25.0505
1,3111,1,1990.0,0.31035,0.19205,0.61870,0.90395,0.17310,0.30105,0.16845,...,11033,3582,7451,-140,11.2765,2.8260,11.4250,3.7090,-0.1480,20.4605
2,3111,2,1992.0,0.29240,0.22650,0.56650,0.71810,0.37265,0.28870,0.18360,...,10527,3263,7264,-628,10.1080,2.4525,10.7490,3.3245,-0.6410,20.2160
3,3111,3,1994.0,0.46585,0.22875,0.79560,1.55100,0.23295,0.34450,0.21130,...,8607,2239,6368,406,9.2985,2.2280,8.8820,2.3100,0.4170,16.5530
4,3111,4,1996.0,0.55025,0.27605,1.02215,1.51450,0.38375,0.38755,0.21315,...,10031,2649,7382,-1376,9.1830,3.0780,10.6295,2.8040,-1.4460,18.3670
5,3111,5,1998.0,0.45445,0.27590,0.78005,1.86950,0.29495,0.31090,0.22045,...,9493,2778,6715,927,11.1790,4.0545,10.1975,2.9860,0.9820,19.0270
6,3111,6,2000.0,0.43095,0.22190,0.59480,1.76500,0.55190,0.28085,0.20160,...,9413,3065,6348,1870,11.5710,2.9570,9.6540,3.1440,1.9170,19.3075
7,3111,7,2002.0,0.55375,0.28385,0.80635,2.18000,0.56580,0.32710,0.25075,...,14728,4419,10309,-3741,11.3295,4.4100,15.1980,4.5590,-3.8685,22.6595
8,3111,8,2004.0,0.73610,0.36300,1.12210,3.01100,0.13120,0.36055,0.22750,...,10963,3314,7649,-293,11.5215,3.0925,11.8315,3.5775,-0.3100,21.0815
9,3111,9,2006.0,0.99065,0.46260,1.82800,2.94150,0.16020,0.37625,0.31540,...,9113,2627,6486,1709,11.4305,3.2445,9.6280,2.7765,1.8025,19.2560


In [150]:
two_yr.describe()

,window_id,window_end_year,sd,d7525,d9010,d9990,d1001,sd*,d7525*,d9010*,...,job_destruction,job_destruction_deaths,job_destruction_continuers,net_job_creation,job_creation_rate,job_creation_rate_births,job_destruction_rate,job_destruction_rate_deaths,net_job_creation_rate,reallocation_rate
count,1462.000000,1462.000000,1462.000000,1462.000000,1462.000000,1461.000000,1462.000000,1462.000000,1462.000000,1462.000000,...,1462.000000,1462.000000,1462.000000,1462.000000,1462.000000,1462.000000,1462.000000,1462.000000,1462.000000,1462.000000
mean,8.000000,2004.000000,0.438629,0.500009,1.049623,0.764578,0.310474,0.361527,0.468077,0.874176,...,36142.787278,10744.370725,25398.416553,-3552.290014,9.613784,2.326393,11.059133,3.268817,-1.445345,16.838861
std,4.900656,9.801312,0.142474,0.203372,0.371008,0.415914,0.148948,0.134410,0.269868,0.353691,...,39050.081954,13312.047692,26787.122804,18513.853423,2.864406,1.173225,4.107130,1.813124,4.515619,4.767277
min,0.000000,1988.000000,0.185150,0.098320,0.429350,0.017920,0.033275,0.139350,0.102355,0.246450,...,578.000000,89.000000,414.000000,-152063.000000,2.437500,0.133500,2.437500,0.165000,-20.519000,4.875000
25%,4.000000,1996.000000,0.345862,0.380150,0.820950,0.480550,0.230875,0.285212,0.331200,0.673675,...,12441.250000,3177.000000,8727.250000,-6534.750000,7.630250,1.530625,8.292250,2.102875,-3.273750,13.833375
50%,8.000000,2004.000000,0.411575,0.454550,0.975350,0.672300,0.295375,0.339675,0.414650,0.812775,...,23284.500000,6483.500000,16533.000000,-879.500000,9.293000,2.102250,10.227250,2.921500,-0.556500,16.464500
75%,12.000000,2012.000000,0.493950,0.553912,1.181250,0.936250,0.369000,0.406138,0.533600,0.996250,...,44335.500000,13048.500000,32126.250000,2727.250000,11.216500,2.860125,12.852500,3.983125,1.259000,19.491500
max,16.000000,2020.000000,1.647500,2.000000,3.861000,3.431000,2.260650,1.573000,3.195500,3.735000,...,298634.000000,130154.000000,210734.000000,68902.000000,24.890500,8.905500,31.406500,14.992000,13.567000,38.216000


In [151]:
two_yr.columns

Index(['naics4', 'window_id', 'window_end_year', 'sd', 'd7525', 'd9010',
       'd9990', 'd1001', 'sd*', 'd7525*', 'd9010*', 'd9990*', 'd1001*',
       'firms', 'estabs', 'emp', 'denom', 'estabs_entry', 'estabs_exit',
       'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp',
       'estabs_entry_rate', 'estabs_exit_rate', 'job_creation',
       'job_creation_births', 'job_creation_continuers', 'job_destruction',
       'job_destruction_deaths', 'job_destruction_continuers',
       'net_job_creation', 'job_creation_rate', 'job_creation_rate_births',
       'job_destruction_rate', 'job_destruction_rate_deaths',
       'net_job_creation_rate', 'reallocation_rate'],
      dtype='object')

In [152]:
# -------------------------------------------------------
# 1. START WITH YOUR AGGREGATED DATA (two_yr)
# -------------------------------------------------------
print("="*60)
print("PREPARING DATA FOR REGRESSION")
print("="*60)

# Create a working copy
df = two_yr.copy()

# -------------------------------------------------------
# 2. CALCULATE GROWTH FOR BOTH IQR AND 90-10 MEASURES
# -------------------------------------------------------

# First make sure the data is sorted properly
df_sorted = df.sort_values(['naics4', 'window_end_year'])

if 'd7525*' in df_sorted.columns:
    # Calculate growth matching paper's formula for IQR with 2-period lag
    df_sorted['iqr_growth_weighted'] = (
        (df_sorted.groupby('naics4')['d7525*'].transform(
            lambda x: (x / x.shift(2))**0.5 - 1
        )) * 100
    )
    
    print(f"IQR Growth with 2-period lag: {df_sorted['iqr_growth_weighted'].describe()}")
    
    # Calculate growth for 90-10 percentile difference with 2-period lag
    if 'd9010*' in df_sorted.columns:
        df_sorted['d9010_growth_weighted'] = (
            (df_sorted.groupby('naics4')['d9010*'].transform(
                lambda x: (x / x.shift(2))**0.5 - 1
            )) * 100
        )
        
        print(f"\n90-10 Growth with 2-period lag: {df_sorted['d9010_growth_weighted'].describe()}")
    else:
        print("\n90-10 weighted variable not found in dataset")
    
    # Use the sorted dataframe
    df = df_sorted.copy()
else:
    print("IQR weighted variable not found in dataset")

PREPARING DATA FOR REGRESSION
IQR Growth with 2-period lag: count    1290.000000
mean        1.096188
std        11.410194
min       -50.090240
25%        -5.031602
50%         0.888540
75%         6.474304
max        55.606575
Name: iqr_growth_weighted, dtype: float64

90-10 Growth with 2-period lag: count    1290.000000
mean        0.904066
std         8.284765
min       -33.137390
25%        -4.052889
50%         0.733789
75%         5.067862
max        45.749277
Name: d9010_growth_weighted, dtype: float64


In [153]:
# -------------------------------------------------------
# 3. CREATE LAGGED ENTRY RATE VARIABLES
# -------------------------------------------------------
# Sort by industry and time to ensure proper lagging
df = df.sort_values(['naics4', 'window_end_year'])

# Create lagged entry rates
df['estabs_entry_rate_lag1'] = df.groupby('naics4')['estabs_entry_rate'].shift(1)
df['estabs_entry_rate_lag2'] = df.groupby('naics4')['estabs_entry_rate'].shift(2)
df['estabs_entry_rate_lag3'] = df.groupby('naics4')['estabs_entry_rate'].shift(3)

print(f"Created lagged entry rate variables")
print(f"Missing values created:")
print(f"  Lag 1: {df['estabs_entry_rate_lag1'].isna().sum()}")
print(f"  Lag 2: {df['estabs_entry_rate_lag2'].isna().sum()}")
print(f"  Lag 3: {df['estabs_entry_rate_lag3'].isna().sum()}")

# -------------------------------------------------------
# 4. CREATE SHORTER NAMES FOR LAGS (for cleaner output)
# -------------------------------------------------------
df['entry_lag1'] = df['estabs_entry_rate_lag1']
df['entry_lag2'] = df['estabs_entry_rate_lag2']
df['entry_lag3'] = df['estabs_entry_rate_lag3']

print(f"\nCreated shorter lag names")

Created lagged entry rate variables
Missing values created:
  Lag 1: 86
  Lag 2: 172
  Lag 3: 258

Created shorter lag names


In [154]:
df.head()

,naics4,window_id,window_end_year,sd,d7525,d9010,d9990,d1001,sd*,d7525*,...,net_job_creation_rate,reallocation_rate,iqr_growth_weighted,d9010_growth_weighted,estabs_entry_rate_lag1,estabs_entry_rate_lag2,estabs_entry_rate_lag3,entry_lag1,entry_lag2,entry_lag3
0,3111,0,1988.0,0.28325,0.21175,0.59035,0.73105,0.14195,0.29455,0.17380,...,-1.025,25.0505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3111,1,1990.0,0.31035,0.19205,0.61870,0.90395,0.17310,0.30105,0.16845,...,-0.148,20.4605,NaN,NaN,6.1835,NaN,NaN,6.1835,NaN,NaN
2,3111,2,1992.0,0.29240,0.22650,0.56650,0.71810,0.37265,0.28870,0.18360,...,-0.641,20.2160,2.780672,-4.990187,5.3165,6.1835,NaN,5.3165,6.1835,NaN
3,3111,3,1994.0,0.46585,0.22875,0.79560,1.55100,0.23295,0.34450,0.21130,...,0.417,16.5530,11.999025,20.300376,5.3335,5.3165,6.1835,5.3335,5.3165,6.1835
4,3111,4,1996.0,0.55025,0.27605,1.02215,1.51450,0.38375,0.38755,0.21315,...,-1.446,18.3670,7.747284,24.864130,4.9785,5.3335,5.3165,4.9785,5.3335,5.3165


In [155]:
df.columns

Index(['naics4', 'window_id', 'window_end_year', 'sd', 'd7525', 'd9010',
       'd9990', 'd1001', 'sd*', 'd7525*', 'd9010*', 'd9990*', 'd1001*',
       'firms', 'estabs', 'emp', 'denom', 'estabs_entry', 'estabs_exit',
       'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp',
       'estabs_entry_rate', 'estabs_exit_rate', 'job_creation',
       'job_creation_births', 'job_creation_continuers', 'job_destruction',
       'job_destruction_deaths', 'job_destruction_continuers',
       'net_job_creation', 'job_creation_rate', 'job_creation_rate_births',
       'job_destruction_rate', 'job_destruction_rate_deaths',
       'net_job_creation_rate', 'reallocation_rate', 'iqr_growth_weighted',
       'd9010_growth_weighted', 'estabs_entry_rate_lag1',
       'estabs_entry_rate_lag2', 'estabs_entry_rate_lag3', 'entry_lag1',
       'entry_lag2', 'entry_lag3'],
      dtype='object')

In [156]:
# -------------------------------------------------------
# 4. CREATE HIGH-TECH INDICATOR
# -------------------------------------------------------
# Define high-tech industries exactly as in the paper
high_tech_naics4 = [3241, 3251, 3252, 3254, 3332, 3333, 3336, 3339, 
                    3341, 3342, 3343, 3344, 3345, 3346, 3353, 3364]

# Convert naics4 to int if it's stored as string
df['naics4_int'] = df['naics4'].astype(int)
df['hightech'] = df['naics4_int'].isin(high_tech_naics4).astype(int)

print(f"\nHigh-tech distribution:")
print(f"Total obs: {len(df)}")
print(f"High-tech: {df['hightech'].sum()} ({df['hightech'].mean()*100:.1f}%)")

# -------------------------------------------------------
# 5. CREATE INTERACTION TERMS
# -------------------------------------------------------
# Using the shorter names
df['entry_lag1_x_hightech'] = df['entry_lag1'] * df['hightech']
df['entry_lag2_x_hightech'] = df['entry_lag2'] * df['hightech']
df['entry_lag3_x_hightech'] = df['entry_lag3'] * df['hightech']

print(f"\nCreated interaction terms")


High-tech distribution:
Total obs: 1462
High-tech: 272 (18.6%)

Created interaction terms


In [157]:
# -------------------------------------------------------
# 6. DROP MISSING VALUES
# -------------------------------------------------------
# Drop rows where we can't calculate growth or don't have all lags
print(f"\nObservations before dropping NAs: {len(df)}")
df_clean = df.dropna(subset=['iqr_growth_weighted', 'entry_lag1', 'entry_lag2', 'entry_lag3'])
print(f"Observations after dropping NAs: {len(df_clean)}")


Observations before dropping NAs: 1462
Observations after dropping NAs: 1204


In [158]:
df_clean.describe()

,window_id,window_end_year,sd,d7525,d9010,d9990,d1001,sd*,d7525*,d9010*,...,estabs_entry_rate_lag2,estabs_entry_rate_lag3,entry_lag1,entry_lag2,entry_lag3,naics4_int,hightech,entry_lag1_x_hightech,entry_lag2_x_hightech,entry_lag3_x_hightech
count,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,...,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000
mean,9.500000,2007.000000,0.451721,0.506223,1.072736,0.816151,0.314261,0.367055,0.471581,0.884711,...,7.369897,7.602091,7.156141,7.369897,7.602091,3266.453488,0.186047,1.317531,1.365768,1.423465
std,4.032804,8.065608,0.146346,0.205910,0.383381,0.431161,0.152814,0.137347,0.275259,0.361222,...,2.674185,2.715202,2.617426,2.674185,2.715202,89.771736,0.389306,2.935403,3.050506,3.190610
min,3.000000,1994.000000,0.185150,0.115060,0.429350,0.017920,0.033275,0.139350,0.110055,0.246450,...,2.457000,2.457000,2.457000,2.457000,2.457000,3111.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,2000.000000,0.360175,0.384550,0.846175,0.527713,0.234550,0.291825,0.334600,0.685150,...,5.574625,5.689375,5.435875,5.574625,5.689375,3169.000000,0.000000,0.000000,0.000000,0.000000
50%,9.500000,2007.000000,0.422700,0.459475,0.999100,0.722700,0.299950,0.343575,0.421300,0.826475,...,6.840000,7.067500,6.644000,6.840000,7.067500,3295.000000,0.000000,0.000000,0.000000,0.000000
75%,13.000000,2014.000000,0.508400,0.555575,1.199700,1.005437,0.370900,0.408275,0.535300,0.996837,...,8.679000,8.962125,8.329250,8.679000,8.962125,3341.000000,0.000000,0.000000,0.000000,0.000000
max,16.000000,2020.000000,1.647500,2.000000,3.861000,3.431000,2.260650,1.573000,3.195500,3.735000,...,21.052000,21.052000,21.052000,21.052000,21.052000,3399.000000,1.000000,15.028000,15.028000,18.135500


In [163]:
print(df_clean.columns)# Convert 'window_end_year' to a numeric data type
df_clean['window_end_year'] = pd.to_numeric(df_clean['window_end_year'], errors='coerce')

# Verify the data type and check for any NaN values introduced during conversion
print(df_clean['window_end_year'].dtype)
print(f"Number of NaN values in 'window_end_year': {df_clean['window_end_year'].isna().sum()}")

Index(['naics4', 'window_id', 'window_end_year', 'sd', 'd7525', 'd9010',
       'd9990', 'd1001', 'sd*', 'd7525*', 'd9010*', 'd9990*', 'd1001*',
       'firms', 'estabs', 'emp', 'denom', 'estabs_entry', 'estabs_exit',
       'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp',
       'estabs_entry_rate', 'estabs_exit_rate', 'job_creation',
       'job_creation_births', 'job_creation_continuers', 'job_destruction',
       'job_destruction_deaths', 'job_destruction_continuers',
       'net_job_creation', 'job_creation_rate', 'job_creation_rate_births',
       'job_destruction_rate', 'job_destruction_rate_deaths',
       'net_job_creation_rate', 'reallocation_rate', 'iqr_growth_weighted',
       'd9010_growth_weighted', 'estabs_entry_rate_lag1',
       'estabs_entry_rate_lag2', 'estabs_entry_rate_lag3', 'entry_lag1',
       'entry_lag2', 'entry_lag3', 'naics4_int', 'hightech',
       'entry_lag1_x_hightech', 'entry_lag2_x_hightech',
       'entry_lag3_x_hightech'],
      dtype='object'

/var/folders/xw/3fpv2dn948lcf41k_wd6vbxh0000gn/T/ipykernel_47945/2722325257.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['window_end_year'] = pd.to_numeric(df_clean['window_end_year'], errors='coerce')


In [159]:
df_final = df_clean.copy()

# Set up panel
df_panel = df_final.set_index(['naics4', 'window_end_year'])

# Define regressors
main_regressors = ['entry_lag1', 'entry_lag2', 'entry_lag3']
interaction_regressors = ['entry_lag1_x_hightech', 'entry_lag2_x_hightech', 'entry_lag3_x_hightech']
all_regressors = main_regressors + interaction_regressors

print(f"\nPanel structure:")
print(f"Entities (naics4): {df_final['naics4'].nunique()}")
print(f"Time periods: {df_final['window_end_year'].nunique()}")
print(f"Total obs: {len(df_panel)}")

print(f"\nReady to run:")
print(f"y = df_panel['iqr_growth_weighted']")
print(f"X = df_panel[{all_regressors}]")
print(f"model = PanelOLS(y, X, entity_effects=True, time_effects=True)")


Panel structure:
Entities (naics4): 86
Time periods: 14
Total obs: 1204

Ready to run:
y = df_panel['iqr_growth_weighted']
X = df_panel[['entry_lag1', 'entry_lag2', 'entry_lag3', 'entry_lag1_x_hightech', 'entry_lag2_x_hightech', 'entry_lag3_x_hightech']]
model = PanelOLS(y, X, entity_effects=True, time_effects=True)


In [129]:
from linearmodels.panel import PanelOLS

# Define your variables
y = df_panel['iqr_growth_weighted']
X = df_panel[['entry_lag1', 'entry_lag2', 'entry_lag3', 
              'entry_lag1_x_hightech', 'entry_lag2_x_hightech', 'entry_lag3_x_hightech']]

# Create the model
model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# Fit with clustering at entity (naics4) level
results = model.fit(cov_type='clustered', cluster_entity=True)

# Display the results
print(results)

# To verify clustering details:
print("\n" + "="*60)
print("REGRESSION DETAILS")
print("="*60)
print(f"Entity dimension: {results.entity_info['total']}")  # Number of naics4 codes
print(f"Time dimension: {results.time_info['total']}")      # Number of time periods
print(f"Total observations: {results.nobs}")
print(f"Covariance type: {results._cov_type}")            # Should show 'clustered'


                           PanelOLS Estimation Summary                           
Dep. Variable:     iqr_growth_weighted   R-squared:                        0.0203
Estimator:                    PanelOLS   R-squared (Between):             -7.6798
No. Observations:                 1204   R-squared (Within):               0.0148
Date:                 Mon, May 26 2025   R-squared (Overall):             -0.4979
Time:                         16:46:26   Log-likelihood                   -4595.2
Cov. Estimator:              Clustered                                           
                                         F-statistic:                      3.7939
Entities:                           86   P-value                           0.0010
Avg Obs:                        14.000   Distribution:                  F(6,1099)
Min Obs:                        14.000                                           
Max Obs:                        14.000   F-statistic (robust):             7.9574
                

In [52]:
# Print regression results in LaTeX format
latex_output = results.summary.as_latex()
print(latex_output)

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}           & iqr\_growth\_weighted & \textbf{  R-squared:         }   &      0.0203      \\
\textbf{Estimator:}               &        PanelOLS       & \textbf{  R-squared (Between):}  &     -7.6798      \\
\textbf{No. Observations:}        &          1204         & \textbf{  R-squared (Within):}   &      0.0148      \\
\textbf{Date:}                    &    Fri, May 23 2025   & \textbf{  R-squared (Overall):}  &     -0.4979      \\
\textbf{Time:}                    &        17:49:12       & \textbf{  Log-likelihood     }   &     -4595.2      \\
\textbf{Cov. Estimator:}          &       Clustered       & \textbf{                     }   &                  \\
\textbf{}                         &                       & \textbf{  F-statistic:       }   &      3.7939      \\
\textbf{Entities:}                &           86          & \textbf{  P-value            }   &      0.0010      \\
\textbf{Avg Obs:}                 

In [53]:
import numpy as np
from scipy import stats

def lincom(results, params, hypothesis_name=""):
    """Calculate linear combination of parameters and test significance"""
    # Calculate the combined coefficient
    beta = sum(results.params[p] for p in params)
    
    # Get variance-covariance matrix
    vcov = results.cov
    
    # Calculate variance of linear combination
    variance = 0
    for i in params:
        for j in params:
            variance += vcov.loc[i, j]
    
    # Calculate standard error
    se = np.sqrt(variance)
    
    # Calculate t-statistic
    t_stat = beta / se
    
    # Calculate p-value (two-tailed)
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), results.df_resid))
    
    print(f"{hypothesis_name}: {beta:.4f} (SE: {se:.4f}, t: {t_stat:.3f}, p-value: {p_value:.4f})")
    return beta, se, t_stat, p_value

# Test the total effect for high-tech industries
print("\nTotal effects for high-tech industries (with significance tests):")

# Lag 1 total effect
lag1_total, lag1_se, lag1_t, lag1_p = lincom(
    results, 
    ['entry_lag1', 'entry_lag1_x_hightech'],
    "Lag 1 (First period after entry)"
)

# Lag 2 total effect
lag2_total, lag2_se, lag2_t, lag2_p = lincom(
    results, 
    ['entry_lag2', 'entry_lag2_x_hightech'],
    "Lag 2 (Second period after entry)"
)

# Lag 3 total effect
lag3_total, lag3_se, lag3_t, lag3_p = lincom(
    results, 
    ['entry_lag3', 'entry_lag3_x_hightech'],
    "Lag 3 (Third period after entry)"
)

# Summarize significance with stars as in academic papers
def add_stars(p_value):
    if p_value < 0.01:
        return "***"
    elif p_value < 0.05:
        return "**"
    elif p_value < 0.10:
        return "*"
    return ""

print("\nSummary of results (academic paper style):")
print(f"Non-tech industries:")
print(f"  Lag 1: {results.params['entry_lag1']:.4f}{add_stars(results.pvalues['entry_lag1'])}")
print(f"  Lag 2: {results.params['entry_lag2']:.4f}{add_stars(results.pvalues['entry_lag2'])}")
print(f"  Lag 3: {results.params['entry_lag3']:.4f}{add_stars(results.pvalues['entry_lag3'])}")

print(f"\nHigh-tech industries (total effect):")
print(f"  Lag 1: {lag1_total:.4f}{add_stars(lag1_p)}")
print(f"  Lag 2: {lag2_total:.4f}{add_stars(lag2_p)}")
print(f"  Lag 3: {lag3_total:.4f}{add_stars(lag3_p)}")


Total effects for high-tech industries (with significance tests):
Lag 1 (First period after entry): 0.5991 (SE: 0.9828, t: 0.610, p-value: 0.5422)
Lag 2 (Second period after entry): -2.5259 (SE: 0.9726, t: -2.597, p-value: 0.0095)
Lag 3 (Third period after entry): -0.0919 (SE: 0.8586, t: -0.107, p-value: 0.9148)

Summary of results (academic paper style):
Non-tech industries:
  Lag 1: -0.7269**
  Lag 2: 0.2653
  Lag 3: 0.1026

High-tech industries (total effect):
  Lag 1: 0.5991
  Lag 2: -2.5259***
  Lag 3: -0.0919


In [54]:
import numpy as np
from scipy import stats

# Make sure 'results' refers to Model 1 (1204 observations)
# If you need to reassign it:
# results = results_model1  # Uncomment if needed

def lincom(results, params, hypothesis_name=""):
    """Calculate linear combination of parameters and test significance"""
    # Calculate the combined coefficient
    beta = sum(results.params[p] for p in params)
    
    # Get variance-covariance matrix
    vcov = results.cov
    
    # Calculate variance of linear combination
    variance = 0
    for i in params:
        for j in params:
            variance += vcov.loc[i, j]
    
    # Calculate standard error
    se = np.sqrt(variance)
    
    # Calculate t-statistic
    t_stat = beta / se
    
    # Calculate p-value (two-tailed)
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), results.df_resid))
    
    return beta, se, t_stat, p_value

# Calculate total effects for high-tech industries
lag1_total, lag1_se, lag1_t, lag1_p = lincom(
    results, 
    ['entry_lag1', 'entry_lag1_x_hightech']
)

lag2_total, lag2_se, lag2_t, lag2_p = lincom(
    results, 
    ['entry_lag2', 'entry_lag2_x_hightech']
)

lag3_total, lag3_se, lag3_t, lag3_p = lincom(
    results, 
    ['entry_lag3', 'entry_lag3_x_hightech']
)

# Function to add significance stars
def add_stars(p_value):
    if p_value < 0.01:
        return "***"
    elif p_value < 0.05:
        return "**"
    elif p_value < 0.10:
        return "*"
    return ""

# Create a formatted table with hypothesis tests
print("=" * 80)
print("               MODEL 1: PRODUCTIVITY DISPERSION GROWTH AND ENTRY RATES")
print("                         (FULL SAMPLE: 1204 OBSERVATIONS)")
print("=" * 80)
print("\nPanel A: Regression Coefficients")
print("-" * 80)
print(f"{'Parameter':<25} {'Coefficient':<15} {'Std. Error':<15} {'t-statistic':<15} {'p-value':<10}")
print("-" * 80)

# Non-tech industries
print(f"{'NON-TECH INDUSTRIES:':<25}")
print(f"{'  Lag 1 Entry':<25} {results.params['entry_lag1']:.4f}{add_stars(results.pvalues['entry_lag1']):<5} {results.std_errors['entry_lag1']:.4f}{' ':<10} {results.tstats['entry_lag1']:.4f}{' ':<10} {results.pvalues['entry_lag1']:.4f}")
print(f"{'  Lag 2 Entry':<25} {results.params['entry_lag2']:.4f}{add_stars(results.pvalues['entry_lag2']):<5} {results.std_errors['entry_lag2']:.4f}{' ':<10} {results.tstats['entry_lag2']:.4f}{' ':<10} {results.pvalues['entry_lag2']:.4f}")
print(f"{'  Lag 3 Entry':<25} {results.params['entry_lag3']:.4f}{add_stars(results.pvalues['entry_lag3']):<5} {results.std_errors['entry_lag3']:.4f}{' ':<10} {results.tstats['entry_lag3']:.4f}{' ':<10} {results.pvalues['entry_lag3']:.4f}")

print(f"\n{'INTERACTIONS:':<25}")
print(f"{'  Lag 1 Entry × Tech':<25} {results.params['entry_lag1_x_hightech']:.4f}{add_stars(results.pvalues['entry_lag1_x_hightech']):<5} {results.std_errors['entry_lag1_x_hightech']:.4f}{' ':<10} {results.tstats['entry_lag1_x_hightech']:.4f}{' ':<10} {results.pvalues['entry_lag1_x_hightech']:.4f}")
print(f"{'  Lag 2 Entry × Tech':<25} {results.params['entry_lag2_x_hightech']:.4f}{add_stars(results.pvalues['entry_lag2_x_hightech']):<5} {results.std_errors['entry_lag2_x_hightech']:.4f}{' ':<10} {results.tstats['entry_lag2_x_hightech']:.4f}{' ':<10} {results.pvalues['entry_lag2_x_hightech']:.4f}")
print(f"{'  Lag 3 Entry × Tech':<25} {results.params['entry_lag3_x_hightech']:.4f}{add_stars(results.pvalues['entry_lag3_x_hightech']):<5} {results.std_errors['entry_lag3_x_hightech']:.4f}{' ':<10} {results.tstats['entry_lag3_x_hightech']:.4f}{' ':<10} {results.pvalues['entry_lag3_x_hightech']:.4f}")

print("\nPanel B: Joint Hypothesis Tests (High-Tech Total Effects)")
print("-" * 80)
print(f"{'Hypothesis':<25} {'Coefficient':<15} {'Std. Error':<15} {'t-statistic':<15} {'p-value':<10}")
print("-" * 80)
print(f"{'Lag 1 Entry + Tech':<25} {lag1_total:.4f}{add_stars(lag1_p):<5} {lag1_se:.4f}{' ':<10} {lag1_t:.4f}{' ':<10} {lag1_p:.4f}")
print(f"{'Lag 2 Entry + Tech':<25} {lag2_total:.4f}{add_stars(lag2_p):<5} {lag2_se:.4f}{' ':<10} {lag2_t:.4f}{' ':<10} {lag2_p:.4f}")
print(f"{'Lag 3 Entry + Tech':<25} {lag3_total:.4f}{add_stars(lag3_p):<5} {lag3_se:.4f}{' ':<10} {lag3_t:.4f}{' ':<10} {lag3_p:.4f}")

print("\nPanel C: Model Information")
print("-" * 80)
print(f"{'Observations:':<25} {results.nobs}")
print(f"{'Industries:':<25} {results.entity_info['total']}")
print(f"{'Time Periods:':<25} {results.time_info['total']}")
print(f"{'R-squared:':<25} {results.rsquared:.4f}")
print(f"{'R-squared (Within):':<25} {results.rsquared_within:.4f}")

print("\nPanel D: Innovation-Experimentation-Shakeout Hypothesis")
print("-" * 80)
print("HYPOTHESIS TEST RESULTS:")
print("-" * 80)
print("1. EXPERIMENTATION PHASE:")
print(f"   H₀: No increase in dispersion after entry in high-tech industries")
print(f"   H₁: Increase in dispersion after entry in high-tech industries")
print(f"   Result: Lag 1 coefficient = {lag1_total:.4f} (SE: {lag1_se:.4f}, p-value: {lag1_p:.4f})")
print(f"   {'REJECTED' if lag1_total > 0 and lag1_p < 0.05 else 'NOT REJECTED' if lag1_total <= 0 else 'PARTIALLY SUPPORTED'} at α = 0.05")

print("\n2. SHAKEOUT PHASE:")
print(f"   H₀: No decrease in dispersion in second period after entry in high-tech industries")
print(f"   H₁: Decrease in dispersion in second period after entry in high-tech industries")
print(f"   Result: Lag 2 coefficient = {lag2_total:.4f} (SE: {lag2_se:.4f}, p-value: {lag2_p:.4f})")
print(f"   {'REJECTED' if lag2_total < 0 and lag2_p < 0.05 else 'NOT REJECTED'} at α = 0.05")

print("\n3. STABILIZATION PHASE:")
print(f"   H₀: Dispersion effect disappears in third period after entry in high-tech industries")
print(f"   H₁: Dispersion effect persists in third period after entry in high-tech industries")
print(f"   Result: Lag 3 coefficient = {lag3_total:.4f} (SE: {lag3_se:.4f}, p-value: {lag3_p:.4f})")
lag3_insig = abs(lag3_t) < stats.t.ppf(0.975, results.df_resid)
print(f"   {'NOT REJECTED' if lag3_insig else 'REJECTED'} at α = 0.05")

print("\n4. DIFFERENT PATTERNS ACROSS INDUSTRY TYPES:")
print(f"   H₀: High-tech and non-tech industries show same dispersion patterns")
print(f"   H₁: High-tech and non-tech industries show different dispersion patterns")
interaction_significant = (results.pvalues['entry_lag1_x_hightech'] < 0.1 or 
                          results.pvalues['entry_lag2_x_hightech'] < 0.1 or 
                          results.pvalues['entry_lag3_x_hightech'] < 0.1)
print(f"   Result: Lag 2 interaction = {results.params['entry_lag2_x_hightech']:.4f} (SE: {results.std_errors['entry_lag2_x_hightech']:.4f}, p-value: {results.pvalues['entry_lag2_x_hightech']:.4f})")
print(f"   {'REJECTED' if interaction_significant else 'NOT REJECTED'} at α = 0.1")

               MODEL 1: PRODUCTIVITY DISPERSION GROWTH AND ENTRY RATES
                         (FULL SAMPLE: 1204 OBSERVATIONS)

Panel A: Regression Coefficients
--------------------------------------------------------------------------------
Parameter                 Coefficient     Std. Error      t-statistic     p-value   
--------------------------------------------------------------------------------
NON-TECH INDUSTRIES:     
  Lag 1 Entry             -0.7269**    0.3159           -2.3008           0.0216
  Lag 2 Entry             0.2653      0.2482           1.0691           0.2853
  Lag 3 Entry             0.1026      0.3006           0.3414           0.7328

INTERACTIONS:            
  Lag 1 Entry × Tech      1.3261      1.0195           1.3007           0.1936
  Lag 2 Entry × Tech      -2.7912***   0.9644           -2.8943           0.0039
  Lag 3 Entry × Tech      -0.1945      0.9236           -0.2106           0.8332

Panel B: Joint Hypothesis Tests (High-Tech Total Effects

Try previous preferred measures of business dynamism

In [114]:
df_panel.head(10)

window_id       sd    d7525    d9010   d9990    d1001  \
naics4 window_end_year                                                          
3111   1994.0                   3  0.46585  0.22875  0.79560  1.5510  0.23295   
       1996.0                   4  0.55025  0.27605  1.02215  1.5145  0.38375   
       1998.0                   5  0.45445  0.27590  0.78005  1.8695  0.29495   
       2000.0                   6  0.43095  0.22190  0.59480  1.7650  0.55190   
       2002.0                   7  0.55375  0.28385  0.80635  2.1800  0.56580   
       2004.0                   8  0.73610  0.36300  1.12210  3.0110  0.13120   
       2006.0                   9  0.99065  0.46260  1.82800  2.9415  0.16020   
       2008.0                  10  0.70835  0.40945  1.05405  2.8995  0.22285   
       2010.0                  11  0.84335  0.34120  1.13915  2.9745  0.16235   
       2012.0                  12  0.68130  0.39245  1.08095  2.0800  0.37620   

                            sd*   d7525*   d9010*   d9990*  ...  \
naics4 window_end_year                                      ...   
3111   1994.0           0.34450  0.21130  0.72180  0.95430  ...   
       1996.0           0.38755  0.21315  0.74915  0.96240  ...   
       1998.0           0.31090  0.22045  0.70320  0.53445  ...   
       2000.0           0.28085  0.20160  0.55600  0.54360  ...   
       2002.0           0.32710  0.25075  0.71495  0.66580  ...   
       2004.0           0.36055  0.22750  0.85260  0.51180  ...   
       2006.0           0.37625  0.31540  0.88865  0.46255  ...   
       2008.0           0.37980  0.28600  0.93500  0.52500  ...   
       2010.0           0.33855  0.32935  0.80825  0.41490  ...   
       2012.0           0.32875  0.34385  0.71355  0.62485  ...   

                        estabs_entry_rate_lag2  estabs_entry_rate_lag3  \
naics4 window_end_year                                                   
3111   1994.0                           5.3165                  6.1835   
       1996.0                           5.3335                  5.3165   
       1998.0                           4.9785                  5.3335   
       2000.0                           4.6725                  4.9785   
       2002.0                           5.8395                  4.6725   
       2004.0                           5.9745                  5.8395   
       2006.0                           7.1335                  5.9745   
       2008.0                           5.7250                  7.1335   
       2010.0                           6.9405                  5.7250   
       2012.0                           5.4750                  6.9405   

                        entry_lag1  entry_lag2  entry_lag3  naics4_int  \
naics4 window_end_year                                                   
3111   1994.0               5.3335      5.3165      6.1835        3111   
       1996.0               4.9785      5.3335      5.3165        3111   
       1998.0               4.6725      4.9785      5.3335        3111   
       2000.0               5.8395      4.6725      4.9785        3111   
       2002.0               5.9745      5.8395      4.6725        3111   
       2004.0               7.1335      5.9745      5.8395        3111   
       2006.0               5.7250      7.1335      5.9745        3111   
       2008.0               6.9405      5.7250      7.1335        3111   
       2010.0               5.4750      6.9405      5.7250        3111   
       2012.0               4.6135      5.4750      6.9405        3111   

                        hightech  entry_lag1_x_hightech  \
naics4 window_end_year                                    
3111   1994.0                  0                    0.0   
       1996.0                  0                    0.0   
       1998.0                  0                    0.0   
       2000.0                  0                    0.0   
       2002.0                  0                    0.0   
       2004.0                  0                    0.0

In [115]:
df_reset = df_panel.reset_index()

In [116]:
df_reset.head(10)

,naics4,window_end_year,window_id,sd,d7525,d9010,d9990,d1001,sd*,d7525*,...,estabs_entry_rate_lag2,estabs_entry_rate_lag3,entry_lag1,entry_lag2,entry_lag3,naics4_int,hightech,entry_lag1_x_hightech,entry_lag2_x_hightech,entry_lag3_x_hightech
0,3111,1994.0,3,0.46585,0.22875,0.79560,1.5510,0.23295,0.34450,0.21130,...,5.3165,6.1835,5.3335,5.3165,6.1835,3111,0,0.0,0.0,0.0
1,3111,1996.0,4,0.55025,0.27605,1.02215,1.5145,0.38375,0.38755,0.21315,...,5.3335,5.3165,4.9785,5.3335,5.3165,3111,0,0.0,0.0,0.0
2,3111,1998.0,5,0.45445,0.27590,0.78005,1.8695,0.29495,0.31090,0.22045,...,4.9785,5.3335,4.6725,4.9785,5.3335,3111,0,0.0,0.0,0.0
3,3111,2000.0,6,0.43095,0.22190,0.59480,1.7650,0.55190,0.28085,0.20160,...,4.6725,4.9785,5.8395,4.6725,4.9785,3111,0,0.0,0.0,0.0
4,3111,2002.0,7,0.55375,0.28385,0.80635,2.1800,0.56580,0.32710,0.25075,...,5.8395,4.6725,5.9745,5.8395,4.6725,3111,0,0.0,0.0,0.0
5,3111,2004.0,8,0.73610,0.36300,1.12210,3.0110,0.13120,0.36055,0.22750,...,5.9745,5.8395,7.1335,5.9745,5.8395,3111,0,0.0,0.0,0.0
6,3111,2006.0,9,0.99065,0.46260,1.82800,2.9415,0.16020,0.37625,0.31540,...,7.1335,5.9745,5.7250,7.1335,5.9745,3111,0,0.0,0.0,0.0
7,3111,2008.0,10,0.70835,0.40945,1.05405,2.8995,0.22285,0.37980,0.28600,...,5.7250,7.1335,6.9405,5.7250,7.1335,3111,0,0.0,0.0,0.0
8,3111,2010.0,11,0.84335,0.34120,1.13915,2.9745,0.16235,0.33855,0.32935,...,6.9405,5.7250,5.4750,6.9405,5.7250,3111,0,0.0,0.0,0.0
9,3111,2012.0,12,0.68130,0.39245,1.08095,2.0800,0.37620,0.32875,0.34385,...,5.4750,6.9405,4.6135,5.4750,6.9405,3111,0,0.0,0.0,0.0


In [117]:
df_reset.columns

Index(['naics4', 'window_end_year', 'window_id', 'sd', 'd7525', 'd9010',
       'd9990', 'd1001', 'sd*', 'd7525*', 'd9010*', 'd9990*', 'd1001*',
       'firms', 'estabs', 'emp', 'denom', 'estabs_entry', 'estabs_exit',
       'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp',
       'estabs_entry_rate', 'estabs_exit_rate', 'job_creation',
       'job_creation_births', 'job_creation_continuers', 'job_destruction',
       'job_destruction_deaths', 'job_destruction_continuers',
       'net_job_creation', 'job_creation_rate', 'job_creation_rate_births',
       'job_destruction_rate', 'job_destruction_rate_deaths',
       'net_job_creation_rate', 'reallocation_rate', 'iqr_growth_weighted',
       'd9010_growth_weighted', 'estabs_entry_rate_lag1',
       'estabs_entry_rate_lag2', 'estabs_entry_rate_lag3', 'entry_lag1',
       'entry_lag2', 'entry_lag3', 'naics4_int', 'hightech',
       'entry_lag1_x_hightech', 'entry_lag2_x_hightech',
       'entry_lag3_x_hightech'],
      dtype='object'

In [119]:
# Create high growth firms share variable

filepath = '/Users/danielseymour/Developer/EC334-Summative/processed_data/hgfs_by_industry_naics4.csv'
high_growth_firms = pd.read_csv(filepath)
high_growth_firms.head()


,year,vcnaics4,pct_high_growth_emp
0,1978,1131,5.738507
1,1978,1132,8.400927
2,1978,1133,18.464760
3,1978,1141,33.797444
4,1978,1142,45.389049


In [120]:
high_growth_firms.dtypes

year                     int64
vcnaics4                 int64
pct_high_growth_emp    float64
dtype: object

In [121]:
df_reset.dtypes

naics4                          object
window_end_year                float64
window_id                        int64
sd                             float64
d7525                          float64
d9010                          float64
d9990                          float64
d1001                          float64
sd*                            float64
d7525*                         float64
d9010*                         float64
d9990*                         float64
d1001*                         float64
firms                            int64
estabs                           int64
emp                              int64
denom                            int64
estabs_entry                     int64
estabs_exit                      int64
firmdeath_firms                float64
firmdeath_estabs               float64
firmdeath_emp                  float64
estabs_entry_rate              float64
estabs_exit_rate               float64
job_creation                     int64
job_creation_births      

In [122]:
# Convert data types to match
# For high_growth_firms: vcnaics4 is already int64, year is int64
# For df_reset: naics4 is object (string), window_end_year is float64

# Option 1: Convert both to numeric
high_growth_firms['vcnaics4'] = high_growth_firms['vcnaics4'].astype(float)
high_growth_firms['year'] = high_growth_firms['year'].astype(float)
df_reset['naics4'] = pd.to_numeric(df_reset['naics4'], errors='coerce')

# Now merge
df_merged = df_reset.merge(
    high_growth_firms,
    left_on=['naics4', 'window_end_year'],
    right_on=['vcnaics4', 'year'],
    how='left'
)

# Drop duplicate columns
df_merged = df_merged.drop(columns=['vcnaics4', 'year'])

In [123]:
df_merged

,naics4,window_end_year,window_id,sd,d7525,d9010,d9990,d1001,sd*,d7525*,...,estabs_entry_rate_lag3,entry_lag1,entry_lag2,entry_lag3,naics4_int,hightech,entry_lag1_x_hightech,entry_lag2_x_hightech,entry_lag3_x_hightech,pct_high_growth_emp
0,3111,1994.0,3,0.46585,0.22875,0.79560,1.55100,0.23295,0.34450,0.21130,...,6.1835,5.3335,5.3165,6.1835,3111,0,0.0,0.0,0.0,1.949509
1,3111,1996.0,4,0.55025,0.27605,1.02215,1.51450,0.38375,0.38755,0.21315,...,5.3165,4.9785,5.3335,5.3165,3111,0,0.0,0.0,0.0,1.645263
2,3111,1998.0,5,0.45445,0.27590,0.78005,1.86950,0.29495,0.31090,0.22045,...,5.3335,4.6725,4.9785,5.3335,3111,0,0.0,0.0,0.0,2.159723
3,3111,2000.0,6,0.43095,0.22190,0.59480,1.76500,0.55190,0.28085,0.20160,...,4.9785,5.8395,4.6725,4.9785,3111,0,0.0,0.0,0.0,2.331276
4,3111,2002.0,7,0.55375,0.28385,0.80635,2.18000,0.56580,0.32710,0.25075,...,4.6725,5.9745,5.8395,4.6725,3111,0,0.0,0.0,0.0,2.524056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,3399,2012.0,12,0.43245,0.42785,0.96275,1.06435,0.35835,0.39735,0.48640,...,9.1685,7.0690,9.6865,9.1685,3399,0,0.0,0.0,0.0,5.458954
1200,3399,2014.0,13,0.42515,0.42050,1.00855,0.88490,0.41645,0.43285,0.56200,...,9.6865,11.2800,7.0690,9.6865,3399,0,0.0,0.0,0.0,3.870521
1201,3399,2016.0,14,0.38380,0.42665,0.89615,0.79195,0.39255,0.40825,0.54655,...,7.0690,9.9690,11.2800,7.0690,3399,0,0.0,0.0,0.0,3.532045
1202,3399,2018.0,15,0.41295,0.39210,0.95680,0.87205,0.38270,0.40835,0.52400,...,11.2800,10.2005,9.9690,11.2800,3399,0,0.0,0.0,0.0,3.597424


In [125]:
# Create share of employment at new firms 

# df is the DataFrame you showed
df_merged['eer'] = (df_merged['job_creation_births'] / df_merged['denom'])*100

In [126]:
df_merged.columns

Index(['naics4', 'window_end_year', 'window_id', 'sd', 'd7525', 'd9010',
       'd9990', 'd1001', 'sd*', 'd7525*', 'd9010*', 'd9990*', 'd1001*',
       'firms', 'estabs', 'emp', 'denom', 'estabs_entry', 'estabs_exit',
       'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp',
       'estabs_entry_rate', 'estabs_exit_rate', 'job_creation',
       'job_creation_births', 'job_creation_continuers', 'job_destruction',
       'job_destruction_deaths', 'job_destruction_continuers',
       'net_job_creation', 'job_creation_rate', 'job_creation_rate_births',
       'job_destruction_rate', 'job_destruction_rate_deaths',
       'net_job_creation_rate', 'reallocation_rate', 'iqr_growth_weighted',
       'd9010_growth_weighted', 'estabs_entry_rate_lag1',
       'estabs_entry_rate_lag2', 'estabs_entry_rate_lag3', 'entry_lag1',
       'entry_lag2', 'entry_lag3', 'naics4_int', 'hightech',
       'entry_lag1_x_hightech', 'entry_lag2_x_hightech',
       'entry_lag3_x_hightech', 'pct_high_growth_emp'

In [168]:
# Create lagged variables for reallocation_rate
df_merged['reallocation_lag1'] = df_merged.groupby('naics4')['reallocation_rate'].shift(1)
df_merged['reallocation_lag2'] = df_merged.groupby('naics4')['reallocation_rate'].shift(2)
df_merged['reallocation_lag3'] = df_merged.groupby('naics4')['reallocation_rate'].shift(3)

# Create interaction terms with hightech
df_merged['reallocation_lag1_x_hightech'] = df_merged['reallocation_lag1'] * df_merged['hightech']
df_merged['reallocation_lag2_x_hightech'] = df_merged['reallocation_lag2'] * df_merged['hightech']
df_merged['reallocation_lag3_x_hightech'] = df_merged['reallocation_lag3'] * df_merged['hightech']

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd

# Create lagged variables for reallocation_rate
df_merged['hgf_lag1'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(1)
df_merged['hgf_lag2'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(2)
df_merged['hgf_lag3'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(3)
df_merged['hgf_lag4'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(4)
df_merged['hgf_lag5'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(5)

# Create interaction terms with hightech
df_merged['hgf_lag1_x_hightech'] = df_merged['hgf_lag1'] * df_merged['hightech']
df_merged['hgf_lag2_x_hightech'] = df_merged['hgf_lag2'] * df_merged['hightech']
df_merged['hgf_lag3_x_hightech'] = df_merged['hgf_lag3'] * df_merged['hightech']

In [170]:
df_merged.head()

,naics4,window_end_year,window_id,sd,d7525,d9010,d9990,d1001,sd*,d7525*,...,hgf_lag3,hgf_lag1_x_hightech,hgf_lag2_x_hightech,hgf_lag3_x_hightech,reallocation_lag1,reallocation_lag2,reallocation_lag3,reallocation_lag1_x_hightech,reallocation_lag2_x_hightech,reallocation_lag3_x_hightech
0,3111,1994.0,3,0.46585,0.22875,0.79560,1.5510,0.23295,0.34450,0.21130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3111,1996.0,4,0.55025,0.27605,1.02215,1.5145,0.38375,0.38755,0.21315,...,NaN,0.0,NaN,NaN,16.5530,NaN,NaN,0.0,NaN,NaN
2,3111,1998.0,5,0.45445,0.27590,0.78005,1.8695,0.29495,0.31090,0.22045,...,NaN,0.0,0.0,NaN,18.3670,16.553,NaN,0.0,0.0,NaN
3,3111,2000.0,6,0.43095,0.22190,0.59480,1.7650,0.55190,0.28085,0.20160,...,1.949509,0.0,0.0,0.0,19.0270,18.367,16.553,0.0,0.0,0.0
4,3111,2002.0,7,0.55375,0.28385,0.80635,2.1800,0.56580,0.32710,0.25075,...,1.645263,0.0,0.0,0.0,19.3075,19.027,18.367,0.0,0.0,0.0


In [181]:
import numpy as np
from scipy import stats
import pandas as pd
import linearmodels as lm

# First, create ALL the lag variables
df_merged['hgf_lag1'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(1)
df_merged['hgf_lag2'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(2)
df_merged['hgf_lag3'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(3)
df_merged['hgf_lag4'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(4)
df_merged['hgf_lag5'] = df_merged.groupby('naics4')['pct_high_growth_emp'].shift(5)

# Create interaction terms with hightech
df_merged['hgf_x_hightech'] = df_merged['pct_high_growth_emp'] * df_merged['hightech']
df_merged['hgf_lag1_x_hightech'] = df_merged['hgf_lag1'] * df_merged['hightech']
df_merged['hgf_lag2_x_hightech'] = df_merged['hgf_lag2'] * df_merged['hightech']
df_merged['hgf_lag3_x_hightech'] = df_merged['hgf_lag3'] * df_merged['hightech']
df_merged['hgf_lag4_x_hightech'] = df_merged['hgf_lag4'] * df_merged['hightech']
df_merged['hgf_lag5_x_hightech'] = df_merged['hgf_lag5'] * df_merged['hightech']

# Create df_model from df_merged
df_model = df_merged.copy()

# Set the index
df_model = df_model.set_index(['naics4', 'window_end_year'])

# Set up the model with contemporaneous and lagged HGF employment plus interactions
exog_vars = ['pct_high_growth_emp', 'hgf_lag1', 'hgf_lag2', 'hgf_lag3', 'hgf_lag4', 'hgf_lag5',
             'hgf_x_hightech', 'hgf_lag1_x_hightech', 'hgf_lag2_x_hightech', 
             'hgf_lag3_x_hightech', 'hgf_lag4_x_hightech', 'hgf_lag5_x_hightech']

model = lm.PanelOLS(
    dependent=df_model['iqr_growth_weighted'],
    exog=df_model[exog_vars],
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
)

# Fit the model
results = model.fit(cov_type='clustered', cluster_entity=True)

# Define the lincom function for calculating total effects
def lincom(results, params):
    """Calculate linear combination of parameters and test significance"""
    beta = sum(results.params[p] for p in params)
    vcov = results.cov
    variance = 0
    for i in params:
        for j in params:
            variance += vcov.loc[i, j]
    se = np.sqrt(variance)
    t_stat = beta / se
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), results.df_resid))
    return beta, se, t_stat, p_value

# Calculate total effects for high-tech industries
total_t, se_t, t_t, p_t = lincom(results, ['pct_high_growth_emp', 'hgf_x_hightech'])
total_lag1, se_lag1, t_lag1, p_lag1 = lincom(results, ['hgf_lag1', 'hgf_lag1_x_hightech'])
total_lag2, se_lag2, t_lag2, p_lag2 = lincom(results, ['hgf_lag2', 'hgf_lag2_x_hightech'])
total_lag3, se_lag3, t_lag3, p_lag3 = lincom(results, ['hgf_lag3', 'hgf_lag3_x_hightech'])
total_lag4, se_lag4, t_lag4, p_lag4 = lincom(results, ['hgf_lag4', 'hgf_lag4_x_hightech'])
total_lag5, se_lag5, t_lag5, p_lag5 = lincom(results, ['hgf_lag5', 'hgf_lag5_x_hightech'])

# Function to add significance stars
def add_stars(p_value):
    if p_value < 0.01:
        return "***"
    elif p_value < 0.05:
        return "**"
    elif p_value < 0.10:
        return "*"
    return ""

print("=" * 80)
print("HIGH GROWTH EMPLOYMENT → PRODUCTIVITY DISPERSION (WITH HIGH-TECH INTERACTIONS)")
print("=" * 80)

# Print results
print("\nRegression Results")
print("-" * 80)
print(f"{'Variable':<35} {'Coefficient':<15} {'Std. Error':<15} {'t-stat':<10} {'p-value':<10}")
print("-" * 80)

# Non-tech industries
print("NON-TECH INDUSTRIES:")
print(f"{'  HGF Employment % (t)':<35} {results.params['pct_high_growth_emp']:.4f}{add_stars(results.pvalues['pct_high_growth_emp']):<5} {results.std_errors['pct_high_growth_emp']:.4f}{' ':<10} {results.tstats['pct_high_growth_emp']:.3f}{' ':<7} {results.pvalues['pct_high_growth_emp']:.4f}")
print(f"{'  HGF Employment % (t-1)':<35} {results.params['hgf_lag1']:.4f}{add_stars(results.pvalues['hgf_lag1']):<5} {results.std_errors['hgf_lag1']:.4f}{' ':<10} {results.tstats['hgf_lag1']:.3f}{' ':<7} {results.pvalues['hgf_lag1']:.4f}")
print(f"{'  HGF Employment % (t-2)':<35} {results.params['hgf_lag2']:.4f}{add_stars(results.pvalues['hgf_lag2']):<5} {results.std_errors['hgf_lag2']:.4f}{' ':<10} {results.tstats['hgf_lag2']:.3f}{' ':<7} {results.pvalues['hgf_l

SyntaxError: unterminated string literal (detected at line 89) (836543022.py, line 89)

In [ ]:
# Prepare data for Model 2 - include lagged reallocation variables
model2_vars = ['reallocation_rate', 
               'hgf_lag1', 'hgf_lag2', 'hgf_lag3',
               'hgf_lag1_x_hightech', 'hgf_lag2_x_hightech', 'hgf_lag3_x_hightech',
               'reallocation_lag1', 'reallocation_lag2', 'reallocation_lag3',
               'reallocation_lag1_x_hightech', 'reallocation_lag2_x_hightech', 'reallocation_lag3_x_hightech',
               'hightech', 'naics4', 'window_end_year']

df_model2 = df_merged[model2_vars].dropna()

# IMPORTANT: Set the index BEFORE creating the model
df_model2 = df_model2.set_index(['naics4', 'window_end_year'])

# Set up the model with HGF and reallocation variables
exog_vars2 = ['hgf_lag1', 'hgf_lag2', 'hgf_lag3', 
              'hgf_lag1_x_hightech', 'hgf_lag2_x_hightech', 'hgf_lag3_x_hightech',
              'reallocation_lag1', 'reallocation_lag2', 'reallocation_lag3',
              'reallocation_lag1_x_hightech', 'reallocation_lag2_x_hightech', 'reallocation_lag3_x_hightech']

model2 = lm.PanelOLS(
    dependent=df_model2['reallocation_rate'],
    exog=df_model2[exog_vars2],
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
)

# Fit the model
results2 = model2.fit(cov_type='clustered', cluster_entity=True)

# Calculate total effects for high-tech industries
# HGF effects
lag1_total_hgf, lag1_se_hgf, lag1_t_hgf, lag1_p_hgf = lincom(
    results2, 
    ['hgf_lag1', 'hgf_lag1_x_hightech']
)

lag2_total_hgf, lag2_se_hgf, lag2_t_hgf, lag2_p_hgf = lincom(
    results2, 
    ['hgf_lag2', 'hgf_lag2_x_hightech']
)

lag3_total_hgf, lag3_se_hgf, lag3_t_hgf, lag3_p_hgf = lincom(
    results2, 
    ['hgf_lag3', 'hgf_lag3_x_hightech']
)

# Reallocation persistence effects
lag1_total_real, lag1_se_real, lag1_t_real, lag1_p_real = lincom(
    results2, 
    ['reallocation_lag1', 'reallocation_lag1_x_hightech']
)

lag2_total_real, lag2_se_real, lag2_t_real, lag2_p_real = lincom(
    results2, 
    ['reallocation_lag2', 'reallocation_lag2_x_hightech']
)

lag3_total_real, lag3_se_real, lag3_t_real, lag3_p_real = lincom(
    results2, 
    ['reallocation_lag3', 'reallocation_lag3_x_hightech']
)


REALLOCATION RATE PERSISTENCE MODEL

Regression Results
--------------------------------------------------------------------------------
Variable                            Coefficient     Std. Error      t-stat     p-value   
--------------------------------------------------------------------------------
NON-TECH INDUSTRIES:               
  Reallocation Rate (t-1)           0.1226**    0.0492           2.492        0.0129
  Reallocation Rate (t-2)           0.0026      0.0378           0.069        0.9449
  Reallocation Rate (t-3)           0.0048      0.0506           0.095        0.9247

INTERACTIONS WITH HIGH-TECH:       
  Reallocation × Tech (t-1)         0.1270*     0.0667           1.903        0.0574
  Reallocation × Tech (t-2)         -0.0217      0.0826           -0.263        0.7925
  Reallocation × Tech (t-3)         0.0537      0.0583           0.921        0.3571

Joint Hypothesis Tests (High-Tech Total Effects)
---------------------------------------------------------